In [36]:
from importlib import reload
import lancedb
import pandas as pd
from dotenv import load_dotenv

load_dotenv("/Users/leon/.env")

# DB specifications
LANCEDB_DIR = "/Users/leon/Documents/study/MA/lancedb"
TABLE_NAME_DOCS = "documents"
TABLE_NAME_CHUNKS = "chunks"

db = lancedb.connect(LANCEDB_DIR)

In [37]:
from utils import io_helpers

reload(io_helpers)


def add_fictional_creation_date(row):
    if str(row["doc_id"]).startswith("400"):
        return pd.to_datetime("2010-01-01")
    if str(row["doc_id"]).startswith("300"):
        return pd.to_datetime("2015-01-01")
    if str(row["doc_id"]).startswith("100"):
        return pd.to_datetime("2005-01-01")
    return pd.to_datetime("2000-01-01")


def get_documents_with_creation_date() -> pd.DataFrame:
    documents = io_helpers.get_documents(read_embeddings=True).drop(columns="original_doc_ids")
    documents["creation_date"] = documents.apply(add_fictional_creation_date, axis=1)
    return documents

In [38]:
documents = get_documents_with_creation_date()

table = db.create_table(TABLE_NAME_DOCS, data=documents, exist_ok=True)
db[TABLE_NAME_DOCS].head(1)

documents

[2025-06-10T21:55:37Z WARN  lance::dataset::scanner] nprobes is not set because nearest has not been called yet


,doc_id,domain,content,embedding,creation_date
0,40,Finance,Acme Government Solutions is a government indu...,"[0.035415836, 0.015197343, 0.08163272, 0.02999...",2000-01-01
1,41,Finance,Entertainment Enterprises Inc. is an entertain...,"[0.050379667, -0.0005295727, 0.04373168, 0.038...",2000-01-01
2,42,Finance,"Advanced Manufacturing Solutions Inc., establi...","[0.023992022, 0.011302027, 0.050733544, 0.0336...",2000-01-01
3,43,Finance,"EcoGuard Solutions, established on April 15, 2...","[0.0594782, 0.01698723, 0.061538648, 0.0547162...",2000-01-01
4,44,Finance,"Green Fields Agriculture Ltd., established on ...","[0.017018009, 0.014309261, 0.082911275, 0.0502...",2000-01-01
...,...,...,...,...,...
166,400116,Law,In a significant legal proceeding at the Cedar...,"[0.028841885, -0.034240857, 0.044584155, 0.031...",2010-01-01
167,400059,Finance,"Retail Emporium, a well-established retail gia...","[0.03652241, 0.02207698, 0.021544848, 0.039533...",2010-01-01
168,300001,Finance,Changes that occurred in senior management of ...,"[-0.004240031, -0.007925675, 0.018725948, 0.03...",2015-01-01
169,300002,Law,Chief judge according to the court judgment of...,"[-0.0014971758, 0.015988875, 0.023612805, 0.02...",2015-01-01


In [39]:
from langchain_community.document_loaders import DataFrameLoader

table_data: pd.DataFrame = table.search().to_pandas()

loader = DataFrameLoader(table_data, page_content_column="content")
lc_documents = loader.load()

[2025-06-10T21:55:37Z WARN  lance::dataset::scanner] nprobes is not set because nearest has not been called yet


In [40]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

lc_chunks = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""], chunk_size=1024, chunk_overlap=128, strip_whitespace=True, keep_separator=False
).split_documents(lc_documents)


print(len(lc_chunks))
for i in range(len(lc_chunks)):
    lc_chunks[i].metadata["chunk_id"] = i
    lc_chunks[i].id = i

1727


In [41]:
from typing import List
from langchain_core.documents.base import Document
import string


def normalize_string(input: str) -> str:
    translator = str.maketrans({p: "" for p in string.punctuation})
    return input.lower().strip().translate(translator)


def flag_outdated_chunk(data: pd.Series, chunks: List[Document]) -> List[Document]:
    documents = get_documents_with_creation_date()
    creation_date1 = documents[documents["doc_id"] == data["id1"]].squeeze()["creation_date"]
    creation_date2 = documents[documents["doc_id"] == data["id2"]].squeeze()["creation_date"]

    if creation_date1 < creation_date2:  # doc2 is newer
        doc_id_old = data["id1"]
        doc_id_new = data["id2"]
        passage_old = data["conflicting_passage_doc1"]
        passage_new = data["conflicting_passage_doc2"]
    else:  # doc1 is newer
        doc_id_old = data["id2"]
        doc_id_new = data["id1"]
        passage_old = data["conflicting_passage_doc2"]
        passage_new = data["conflicting_passage_doc1"]

    chunk_ids_old = []
    chunk_ids_new = []

    for chunk in chunks:
        if chunk.metadata["doc_id"] not in [doc_id_old, doc_id_new]:
            continue
        if normalize_string(passage_old) in normalize_string(chunk.page_content):
            chunk_ids_old.append(chunk.metadata["chunk_id"])
        elif normalize_string(passage_new) in normalize_string(chunk.page_content):
            chunk_ids_new.append(chunk.metadata["chunk_id"])

    for chunk in chunks:
        if chunk.metadata["chunk_id"] in chunk_ids_old:
            chunk.metadata["outdated_by_chunk_ids"] = chunk_ids_new

    return chunks


conflicts = pd.read_csv(
    "data/additional_data/docs/_conflicts.csv",
    usecols=["id1", "id2", "model", "conflicting_passage_doc1", "conflicting_passage_doc2"],
    dtype={"id1": "Int64", "id2": "Int64"},
)

for _, data in conflicts.iterrows():
    lc_chunks = flag_outdated_chunk(data, lc_chunks)

In [42]:
from langchain.vectorstores import LanceDB
from langchain.embeddings import OpenAIEmbeddings
from openai import RateLimitError
from tenacity import retry, retry_if_exception_type, wait_random, stop_after_attempt

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

try:
    vector_store = LanceDB(uri=LANCEDB_DIR, embedding=embeddings, table=db.open_table(TABLE_NAME_CHUNKS))
except ValueError:
    print("Table not existent")
    vector_store = LanceDB(uri=LANCEDB_DIR, embedding=embeddings, table_name=TABLE_NAME_CHUNKS)

ids_processed = []

In [43]:
print(lc_chunks[0].metadata)
print(lc_chunks[1].metadata)

{'doc_id': 40, 'domain': 'Finance', 'embedding': array([ 0.03541584,  0.01519734,  0.08163272, ..., -0.01773023,
        0.01715001, -0.00257613]), 'creation_date': Timestamp('2000-01-01 00:00:00'), 'chunk_id': 0, 'outdated_by_chunk_ids': [1652]}
{'doc_id': 40, 'domain': 'Finance', 'embedding': array([ 0.03541584,  0.01519734,  0.08163272, ..., -0.01773023,
        0.01715001, -0.00257613]), 'creation_date': Timestamp('2000-01-01 00:00:00'), 'chunk_id': 1}


In [ ]:
@retry(
    retry=retry_if_exception_type(RateLimitError),
    wait=wait_random(min=30, max=60),
    stop=stop_after_attempt(6),
)
def add_chunk_to_vector_store(id, text, metadata):
    if id in ids_processed:
        print(f"Skipping ID '{id}'")
    vector_store.add_texts([text], [metadata], [id])
    ids_processed.append(id)


for chunk in lc_chunks:
    id = chunk.metadata["chunk_id"]
    text = chunk.page_content
    metadata = chunk.metadata.copy()
    if metadata.get("outdated_by_chunk_ids", None) is None:
        metadata["outdated_by_chunk_ids"] = []
    try:
        del metadata["embedding"]
    except KeyError:
        pass
    try:
        del metadata["chunk_id"]
    except KeyError:
        pass
    # add_chunk_to_vector_store(id, text, metadata)

Skipping ID '0'
Skipping ID '1'
Skipping ID '2'


In [56]:
result = vector_store.similarity_search("Medical", k=5)

print(len(result))
print(result)

1
[Document(metadata={'creation_date': datetime.datetime(2015, 1, 1, 0, 0), 'doc_id': 300003, 'domain': 'Medical', 'outdated_by_chunk_ids': []}, page_content='Chief complaint of J. Reyes according to the hospitalization records of Bridgewater General Hospital | Severe headaches and dizziness for 3 months.\nAdmission time for X. Price according to the hospitalization records of Tremont City Hospital | 15th, February')]


In [57]:
chunks_table = db["chunks"].search().to_pandas()
print(len(chunks_table))
print(chunks_table)

1
                                              vector    id  \
0  [-0.016182736, -0.0001533637, 0.0136298835, 0....  1726   

                                                text  \
0  Chief complaint of J. Reyes according to the h...   

                                            metadata  
0  {'creation_date': 2015-01-01 00:00:00, 'doc_id...  


[2025-06-10T22:28:05Z WARN  lance::dataset::scanner] nprobes is not set because nearest has not been called yet


In [ ]:
print(type(db[TABLE_NAME_CHUNKS]))

<class 'lancedb.table.LanceTable'>


In [ ]:
from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector

from tenacity import (
    retry,
    wait_random_exponential,
    retry_if_exception_type,
    stop_after_attempt,
)